Objective: Implement various SVD algos on movielens via tensorflow

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
header = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('data/ml-100k/u.data', sep='\t', names=header)

In [2]:
#re-assign user id starting from 0
user_id_dict = {} 
for user_id in df['user_id']:
    if (user_id) not in user_id_dict.keys():
        user_id_dict[(user_id)] = len(user_id_dict)

        
#re-assign item id starting from 0
item_id_dict = {} 
for item_id in df['item_id']:
    if (item_id) not in item_id_dict.keys():
        item_id_dict[(item_id)] = len(item_id_dict)


In [3]:
# Create features X and labels y
df2 = df.to_numpy()
dat = df2[:,[0,1]]
X = [[user_id_dict[(dat[i,0])], item_id_dict[(dat[i,1])]] for i in range(dat.shape[0])]
X = np.array(X, dtype = np.int32)
y = df2[:,2].astype(np.float32)
y = np.expand_dims(y, 1)


# Extract implicit features
rated_by_user = [[] for i in range(len(user_id_dict))]
for rate in X:
    user = rate[0]
    item = rate[1]
    rated_by_user[user].append(item)


In [94]:
# Vanilla SVD with global effects 

class Vanilla_SVD_layer(tf.keras.layers.Layer):
    def __init__(self, num_features, num_users, num_items):
        super(Vanilla_SVD_layer, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.num_features = num_features

    def build(self, input_shape):
        self.global_bias = self.add_weight("global_bias",
                                            shape=[1, 1]
                                          )
        self.user_vectors = self.add_weight("user_vectors",
                                      shape=[int(self.num_users), int(self.num_features)]
                                           )
        self.item_vectors = self.add_weight("item_vectors",
                                      shape=[int(self.num_items), int(self.num_features)]
                                           )
        self.user_bias = self.add_weight("user_bias",
                                        shape=[int(self.num_users), 1]
                                        )
        self.item_bias = self.add_weight("item_bias",
                                        shape=[int(self.num_items), 1]
                                        )

    def call(self, input):
        user_ids = tf.expand_dims(input[:,0],1)
        item_ids = tf.expand_dims(input[:,1],1)

        user_vectors_input = tf.gather_nd(
        self.user_vectors, user_ids, batch_dims=0, name=None
        )
        item_vectors_input = tf.gather_nd(
        self.item_vectors, item_ids, batch_dims=0, name=None
        )
        user_bias_input = tf.gather_nd(
        self.user_bias, user_ids, batch_dims=0, name=None
        )
        item_bias_input = tf.gather_nd(
        self.item_bias, user_ids, batch_dims=0, name=None
        )
        
        pred = tf.math.reduce_sum(tf.math.multiply(user_vectors_input,item_vectors_input), axis = 1, keepdims = True)

        return (pred + user_bias_input + item_bias_input +  self.global_bias)



class SVD(tf.keras.Model):

    def __init__(self):
        super(SVD, self).__init__()
        self.svd = Vanilla_SVD_layer(5, 10000, 10000)

    def call(self, inputs):
        x = self.svd(inputs)
        return (x)


model = SVD()
model.compile(optimizer="Adam", loss="mse", metrics=[tf.keras.metrics.RootMeanSquaredError()
])

# Check if the model forward pass is working properly. 
model(X[range(10),:])


<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[-0.4069322 ],
       [-0.38670728],
       [-0.4209532 ],
       [-0.38684046],
       [-0.3857712 ],
       [-0.39954796],
       [-0.40087467],
       [-0.35173818],
       [-0.4200813 ],
       [-0.39258903]], dtype=float32)>

In [95]:
model.summary()

Model: "svd_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vanilla_svd_layer_2 (Vanilla multiple                  120001    
Total params: 120,001
Trainable params: 120,001
Non-trainable params: 0
_________________________________________________________________


In [98]:
# Callback for autostopping
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_root_mean_squared_error', patience=5)


# Using Vanilla SVD as Baseline Measure
model.fit(X, y, epochs=100, batch_size=64, validation_split = 0.2, callbacks = [callback])

Epoch 1/100
1250/1250 [==============================] - 1s 778us/step - loss: 0.5972 - root_mean_squared_error: 0.7728 - val_loss: 0.9505 - val_root_mean_squared_error: 0.9749
Epoch 2/100
1250/1250 [==============================] - 1s 736us/step - loss: 0.5971 - root_mean_squared_error: 0.7727 - val_loss: 0.9526 - val_root_mean_squared_error: 0.9760
Epoch 3/100
1250/1250 [==============================] - 1s 719us/step - loss: 0.5967 - root_mean_squared_error: 0.7724 - val_loss: 0.9539 - val_root_mean_squared_error: 0.9767
Epoch 4/100
1250/1250 [==============================] - 1s 718us/step - loss: 0.5968 - root_mean_squared_error: 0.7725 - val_loss: 0.9544 - val_root_mean_squared_error: 0.9769
Epoch 5/100
1250/1250 [==============================] - 1s 722us/step - loss: 0.5963 - root_mean_squared_error: 0.7722 - val_loss: 0.9551 - val_root_mean_squared_error: 0.9773
Epoch 6/100
1250/1250 [==============================] - 1s 709us/step - loss: 0.5963 - root_mean_squared_error: 0.

In [42]:
# SVDPP used in netflix prize

from tensorflow.keras import regularizers

class SVDPP_layer(tf.keras.layers.Layer):
    def __init__(self, num_features, num_users, num_items, l1 = 0, l2 = 0):
        super(SVDPP_layer, self).__init__()
        self._supports_ragged_inputs = True 
        self.num_users = num_users
        self.num_items = num_items
        self.num_features = num_features
        self.l1 = l1
        self.l2 = l2
        
        
    def build(self, input_shape):
        
        self.global_bias = self.add_weight("global_bias",
                                           shape=[1, 1]
                                          )
        self.user_vectors = self.add_weight("user_vectors",
                                            shape=[int(self.num_users), int(self.num_features)],
                                            regularizer=tf.keras.regularizers.l1_l2(l1=self.l1, l2=self.l2)
                                           )
        self.item_vectors = self.add_weight("item_vectors",
                                            shape=[int(self.num_items), int(self.num_features)],
                                            regularizer=tf.keras.regularizers.l1_l2(l1=self.l1, l2=self.l2)
                                           )
        self.item_vectors_2 = self.add_weight("item_vectors_2",
                                              shape=[int(self.num_items), int(self.num_features)],
                                              regularizer=tf.keras.regularizers.l1_l2(l1=self.l1, l2=self.l2)
                                             )
        self.user_bias = self.add_weight("user_bias",
                                         shape=[int(self.num_users + 1), 1]
                                        )
        self.item_bias = self.add_weight("item_bias",
                                         shape=[int(self.num_items + 1), 1]
                                        )
    
    

    def call(self, input):
        user_item = input[:,0:2].to_tensor()
        num_rated_user = tf.cast(tf.squeeze(input[:,2:3].to_tensor()), tf.float32)
        rated_by_user = input[:,3:]
        user_ids = tf.expand_dims(user_item[:,0],1)
        item_ids = tf.expand_dims(user_item[:,1],1)
        
        
        
        user_vectors_input = tf.gather_nd(
        self.user_vectors, user_ids, batch_dims=0, name=None
        )
        item_vectors_input = tf.gather_nd(
        self.item_vectors, item_ids, batch_dims=0, name=None
        )
        user_bias_input = tf.gather_nd(
        self.user_bias, user_ids, batch_dims=0, name=None
        )
        item_bias_input = tf.gather_nd(
        self.item_bias, user_ids, batch_dims=0, name=None
        )
        
        
        
        rated_by_user = tf.expand_dims(rated_by_user,2)
        
        rated_matrix = tf.gather_nd(self.item_vectors_2,rated_by_user, batch_dims=0)
        
        rated_matrix_sum = tf.reduce_sum(rated_matrix,1)
        
        neigh_vector = tf.multiply(rated_matrix_sum, tf.expand_dims(1/tf.math.sqrt(num_rated_user),1))
        
        total_vector = neigh_vector + user_vectors_input
        
        pred = tf.math.reduce_sum(tf.math.multiply(total_vector,item_vectors_input), axis = 1, keepdims = True)
        
        res = user_bias_input + item_bias_input +  self.global_bias + pred
        return (res)

    

class SVDPP(tf.keras.Model):

    def __init__(self):
        super(SVDPP, self).__init__()
        self._supports_ragged_inputs = True #Support ragged tensor
        self.svdpp = SVDPP_layer(5, 10000, 10000, l2=0.0001)

    def call(self, inputs):
        x = self.svdpp(inputs)
        return (x)




In [48]:
# Create ragged feature columns
X_with_implicit_features_10 = [ 
    [X[i,0], 
     X[i,1], 
     max(len(rated_by_user[X[i,0]])-1,1), 
     *[n for n in rated_by_user[X[i,0]] if n != X[i,1]] # Make sure X[i,1] is not in implicit features
    ] 
    for i in range(10)
]

# Convert to ragged tensor 
X_with_implicit_features_10_ragged = tf.ragged.constant(X_with_implicit_features_10)



model = SVDPP()
model.compile(optimizer="Adam", loss="mse", metrics=[tf.keras.metrics.RootMeanSquaredError()])
print(model(X_with_implicit_features_10_ragged))
model.summary()

# Callback for autostopping
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_root_mean_squared_error', patience=5)


tf.Tensor(
[[-0.5752639 ]
 [-0.56866264]
 [-0.60082906]
 [-0.56856346]
 [-0.5675902 ]
 [-0.57178676]
 [-0.59123176]
 [-0.6065353 ]
 [-0.60107464]
 [-0.58581024]], shape=(10, 1), dtype=float32)
Model: "svdpp_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
svdpp_layer_4 (SVDPP_layer)  multiple                  170003    
Total params: 170,003
Trainable params: 170,003
Non-trainable params: 0
_________________________________________________________________


In [49]:
# Create ragged feature columns
X_with_implicit_features = [ 
    [X[i,0], 
     X[i,1], 
     max(len(rated_by_user[X[i,0]])-1,1), 
     *[n for n in rated_by_user[X[i,0]] if n != X[i,1]] # Make sure X[i,1] is not in implicit features
    ] 
    for i in range(100000)
]

X_with_implicit_features_ragged = tf.ragged.constant(X_with_implicit_features) # Convert to ragged tensor

In [91]:
# sklearn's train_test_split doesn't work for ragged tensor
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_with_implicit_features, y, test_size=0.2, random_state=999)
X_train_ragged = tf.ragged.constant(X_train)
X_test_ragged = tf.ragged.constant(X_test)

In [92]:
model.fit(X_train_ragged, y_train, epochs=100, batch_size = 100, verbose = 1, validation_data = (X_test_ragged, y_test), callbacks = [callback])

Epoch 1/100
800/800 [==============================] - 4s 4ms/step - loss: 10.3975 - root_mean_squared_error: 3.1577 - val_loss: 1.8657 - val_root_mean_squared_error: 1.3357
Epoch 2/100
800/800 [==============================] - 3s 3ms/step - loss: 1.6092 - root_mean_squared_error: 1.2342 - val_loss: 1.2902 - val_root_mean_squared_error: 1.0947
Epoch 3/100
800/800 [==============================] - 3s 3ms/step - loss: 1.1770 - root_mean_squared_error: 1.0409 - val_loss: 1.1251 - val_root_mean_squared_error: 1.0153
Epoch 4/100
800/800 [==============================] - 3s 3ms/step - loss: 1.0533 - root_mean_squared_error: 0.9795 - val_loss: 1.0652 - val_root_mean_squared_error: 0.9872
Epoch 5/100
800/800 [==============================] - 3s 3ms/step - loss: 1.0078 - root_mean_squared_error: 0.9586 - val_loss: 1.0193 - val_root_mean_squared_error: 0.9672
Epoch 6/100
800/800 [==============================] - 3s 4ms/step - loss: 0.9668 - root_mean_squared_error: 0.9407 - val_loss: 1.0042

800/800 [==============================] - 3s 3ms/step - loss: 0.7391 - root_mean_squared_error: 0.8213 - val_loss: 0.8759 - val_root_mean_squared_error: 0.9005
Epoch 49/100
800/800 [==============================] - 3s 3ms/step - loss: 0.7474 - root_mean_squared_error: 0.8259 - val_loss: 0.8748 - val_root_mean_squared_error: 0.8997
Epoch 50/100
800/800 [==============================] - 3s 3ms/step - loss: 0.7252 - root_mean_squared_error: 0.8120 - val_loss: 0.8756 - val_root_mean_squared_error: 0.8999
Epoch 51/100
800/800 [==============================] - 3s 3ms/step - loss: 0.7320 - root_mean_squared_error: 0.8160 - val_loss: 0.8767 - val_root_mean_squared_error: 0.9003
Epoch 52/100
800/800 [==============================] - 3s 3ms/step - loss: 0.7334 - root_mean_squared_error: 0.8165 - val_loss: 0.8764 - val_root_mean_squared_error: 0.8998
Epoch 53/100
800/800 [==============================] - 3s 3ms/step - loss: 0.7335 - root_mean_squared_error: 0.8163 - val_loss: 0.8765 - val_r